In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import GPT2Tokenizer, TFGPT2Model
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
df = pd.read_csv("spam.csv",encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
#delete column 
df = df.drop('Unnamed: 2', axis=1)
df = df.drop('Unnamed: 3', axis=1)
df = df.drop('Unnamed: 4', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [4]:
#change column name
df.rename(columns={'v1': 'type', 'v2': 'text'}, inplace=True)

print(df)

      type                                               text
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham              Will Ì_ b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [5]:
# encoding the type to 0,1
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df['type'])

df['label'] = le.transform(df['type'])
df.head()

,type,text,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [8]:
# Split the dataset into training and testing sets.
X = df['text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Initialize and tokenize the GPT-2 model and tokenizer.
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = TFGPT2Model.from_pretrained("gpt2")

All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [10]:
# Initialize and tokenize the GPT-2 model and tokenizer.
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add a padding token
gpt2_model = TFGPT2Model.from_pretrained("gpt2")

All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [11]:
# Tokenize and pad the SMS messages using GPT-2 tokenizer.
max_sequence_length = 128
X_train_tokens = gpt2_tokenizer(list(X_train), truncation=True, padding=True, max_length=max_sequence_length, return_tensors='tf')
X_test_tokens = gpt2_tokenizer(list(X_test), truncation=True, padding=True, max_length=max_sequence_length, return_tensors='tf')

In [16]:
# Build the LSTM model
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(input_dim=len(gpt2_tokenizer), output_dim=100, input_length=max_sequence_length))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [17]:
# Compile the model.
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
# Train the model.
batch_size = 32
epochs = 5 
model.fit(X_train_tokens['input_ids'], y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/5
112/112 [==============================] - 10s 88ms/step - loss: 0.3964 - accuracy: 0.8670 - val_loss: 0.4012 - val_accuracy: 0.8621
Epoch 2/5
112/112 [==============================] - 10s 90ms/step - loss: 0.3979 - accuracy: 0.8670 - val_loss: 0.4010 - val_accuracy: 0.8621
Epoch 3/5
112/112 [==============================] - 10s 91ms/step - loss: 0.3985 - accuracy: 0.8670 - val_loss: 0.4010 - val_accuracy: 0.8621
Epoch 4/5
112/112 [==============================] - 10s 88ms/step - loss: 0.4004 - accuracy: 0.8670 - val_loss: 0.4013 - val_accuracy: 0.8621
Epoch 5/5
112/112 [==============================] - 9s 84ms/step - loss: 0.3998 - accuracy: 0.8670 - val_loss: 0.4024 - val_accuracy: 0.8621


In [26]:
# Evaluate the model.
loss, accuracy = model.evaluate(X_test_tokens['input_ids'], y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

35/35 [==============================] - 1s 23ms/step - loss: 0.3949 - accuracy: 0.8655
Test loss: 0.3949, Test accuracy: 0.8655
